In [1]:
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output
from dataclasses import dataclass
from typing import List, Tuple, Optional
import numpy.linalg as LA

In [2]:
@dataclass
class Vertex:
    x: float
    y: float
    z: float
    id: int
    
    def to_array(self) -> np.ndarray:
        return np.array([self.x, self.y, self.z])

class PolyhedralSurface:
    def __init__(self):
        self.vertices: List[Vertex] = []
        self.faces: List[List[int]] = []
        
    def add_vertex(self, x: float, y: float, z: float) -> int:
        vid = len(self.vertices)
        self.vertices.append(Vertex(x, y, z, vid))
        return vid
    
    def add_face(self, vertex_indices: List[int]) -> None:
        self.faces.append(vertex_indices)
        
    def get_vertex_coordinates(self) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
        coords = np.array([v.to_array() for v in self.vertices])
        return coords[:, 0], coords[:, 1], coords[:, 2]
    
    def get_face_coordinates(self) -> Tuple[List[float], List[float], List[float]]:
        x, y, z = [], [], []
        for face in self.faces:
            for idx in face:
                vertex = self.vertices[idx]
                x.append(vertex.x)
                y.append(vertex.y)
                z.append(vertex.z)
            x.append(None)
            y.append(None)
            z.append(None)
        return x, y, z

In [3]:
class GeodesicVisualization:
    def __init__(self):
        self.surface = self._create_sample_surface()
        self.current_path = []
        self.highlighted_face = -1
        self.fig = None
        self.step_count = 0
        
    def _create_sample_surface(self) -> PolyhedralSurface:
        surface = PolyhedralSurface()
        
        # Add vertices for a pyramid
        apex = surface.add_vertex(0, 2, 0)
        v1 = surface.add_vertex(-1, 0, -1)
        v2 = surface.add_vertex(1, 0, -1)
        v3 = surface.add_vertex(1, 0, 1)
        v4 = surface.add_vertex(-1, 0, 1)
        
        # Add faces
        surface.add_face([apex, v1, v2])
        surface.add_face([apex, v2, v3])
        surface.add_face([apex, v3, v4])
        surface.add_face([apex, v4, v1])
        surface.add_face([v1, v2, v3, v4])
        
        return surface
    
    def create_figure(self) -> go.Figure:
        x, y, z = self.surface.get_vertex_coordinates()
        faces_x, faces_y, faces_z = self.surface.get_face_coordinates()
        
        self.fig = go.Figure()
        
        self.fig.add_trace(go.Mesh3d(
            x=x, y=y, z=z,
            i=[f[0] for f in self.surface.faces],
            j=[f[1] for f in self.surface.faces],
            k=[f[2] for f in self.surface.faces],
            opacity=0.7,
            color='lightblue',
            name='Surface'
        ))
        
        self.fig.add_trace(go.Scatter3d(
            x=x, y=y, z=z,
            mode='markers',
            marker=dict(size=8, color='blue'),
            name='Vertices'
        ))
        
        self.fig.update_layout(
            scene=dict(
                aspectmode='data',
                camera=dict(
                    up=dict(x=0, y=1, z=0),
                    center=dict(x=0, y=0, z=0),
                    eye=dict(x=1.5, y=1.5, z=1.5)
                )
            ),
            showlegend=True,
            title='Geodesic Path Visualization',
            height=800
        )
        
        return self.fig
    
    def update_visualization(self, path: List[int], highlighted_face: int) -> None:
        if not self.fig:
            return
            
        if path:
            path_vertices = [self.surface.vertices[i] for i in path]
            path_x = [v.x for v in path_vertices]
            path_y = [v.y for v in path_vertices]
            path_z = [v.z for v in path_vertices]
            
            path_trace = go.Scatter3d(
                x=path_x, y=path_y, z=path_z,
                mode='lines+markers',
                line=dict(color='red', width=4),
                marker=dict(size=8, color='red'),
                name='Current Path'
            )
            
            self.fig.data = list(self.fig.data[:2])
            self.fig.add_trace(path_trace)
            
        if highlighted_face >= 0:
            face_vertices = [self.surface.vertices[i] for i in self.surface.faces[highlighted_face]]
            face_x = [v.x for v in face_vertices]
            face_y = [v.y for v in face_vertices]
            face_z = [v.z for v in face_vertices]
            
            highlight_trace = go.Mesh3d(
                x=face_x, y=face_y, z=face_z,
                opacity=0.9,
                color='pink',
                name='Highlighted Face'
            )
            self.fig.add_trace(highlight_trace)

In [4]:
def create_interactive_visualization():
    vis = GeodesicVisualization()
    fig = vis.create_figure()
    
    play_button = widgets.Play(
        value=0,
        min=0,
        max=4,
        step=1,
        interval=1000,
        description="Play",
        disabled=False
    )
    
    slider = widgets.IntSlider(
        value=0,
        min=0,
        max=4,
        step=1,
        description='Step:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True
    )
    
    reset_button = widgets.Button(
        description='Reset',
        disabled=False,
        button_style='warning',
        tooltip='Reset visualization'
    )
    
    widgets.jslink((play_button, 'value'), (slider, 'value'))
    
    steps = [
        ([], -1),
        ([0], 0),
        ([0, 2], 0),
        ([0, 2, 3], 1),
        ([0, 2, 3], -1)
    ]
    
    def update(change):
        step_index = change['new']
        path, face = steps[step_index]
        vis.update_visualization(path, face)
        fig.show()
    
    def reset(_):
        slider.value = 0
        play_button.value = 0
    
    slider.observe(update, names='value')
    reset_button.on_click(reset)
    
    controls = widgets.HBox([play_button, slider, reset_button])
    display(controls)
    display(fig)
    
    return vis, fig

In [5]:
vis, fig = create_interactive_visualization()